In [ ]:
!pip install segmentation_models_pytorch

In [ ]:
!pip install rasterio

In [ ]:
!pip install torch

In [ ]:
!pip install patchify

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import os
os.environ['TF_ENABLE-ONEDNN-OPTS'] ='0'
import torch
import numpy as np
from  matplotlib  import pyplot as plt
from patchify import patchify, unpatchify
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
import torch
import numpy as np
from matplotlib import pyplot as plt
from patchify import patchify, unpatchify
from PIL import Image
from sklearn.preprocessing import MinMaxScaler
from segmentation_models_pytorch import Unet
import segmentation_models_pytorch.utils
import torch
import rasterio

In [ ]:
# Function

#to convert label to RGB

def label_to_rgb(mask):
    background = np.array([0, 0, 0])  # Background
    building = np.array([60, 16, 152])  # Building
    woodland = np.array([132, 41, 246])  # Woodland
    water = np.array([226, 169, 41])  # Water

    segmented_img = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
    segmented_img[mask == 0] = background
    segmented_img[mask == 1] = building
    segmented_img[mask == 2] = woodland
    segmented_img[mask == 3] = water
    return segmented_img

In [ ]:

image_path = "/content/drive/MyDrive/Semantic_segmentation_dataset2/images/M-33-32-B-b-4-4.tif"
with rasterio.open(image_path) as src:
    img = src.read()
    img = np.moveaxis(img, 0, -1)  # Convert from (bands, rows, cols) to (rows, cols, bands)
    transform = src.transform  # Save the transform information
    crs = src.crs  # Save the CRS information

In [ ]:

model_path = "/content/drive/MyDrive/model/trained_landcover_unet_efficientnet-b0_epochs18_patch512_batch16 (2).pth"
model = torch.load(model_path, map_location=torch.device('cpu'))
model.eval()  # Set the model to evaluation mode


In [ ]:
# Function to make predictions
def predict_patch(model, patch):
    patch = torch.tensor(patch).permute(2, 0, 1).unsqueeze(0).float()  # Convert to torch tensor and add batch dimension
    with torch.no_grad():
        pred = model(patch)
    pred = torch.argmax(pred, dim=1).squeeze().numpy()
    return pred

In [ ]:

def predict_patch(model, patch):
    patch = torch.tensor(patch).permute(2, 0, 1).unsqueeze(0).float()  # Convert to torch tensor and add batch dimension
    with torch.no_grad():
        pred = model(patch)
    pred = torch.argmax(pred, dim=1).squeeze().numpy()
    return pred
if img.dtype != np.uint8:
    img = img.astype(np.uint8)
if img.ndim == 2:  # Grayscale image
    img = np.stack((img,)*3, axis=-1)  # Convert to RGB by duplicating channels
elif img.ndim == 3 and img.shape[2] == 1:  # Single channel grayscale image
    img = np.concatenate([img]*3, axis=2)  # Convert to RGB

patch_size = 512
SIZE_X = (img.shape[1] // patch_size) * patch_size
SIZE_Y = (img.shape[0] // patch_size) * patch_size


large_img = img[:SIZE_Y, :SIZE_X]

# Convert to PIL Image and back to numpy array to ensure compatibility
large_img_pil = Image.fromarray(large_img, 'RGB')
large_img = np.array(large_img_pil)

patches_img = patchify(large_img, (patch_size, patch_size, 3), step=patch_size)
patches_img = patches_img[:, :, 0, :, :, :]


In [ ]:

scaler = MinMaxScaler()
# Predict patch by patch
patched_prediction = []
for i in range(patches_img.shape[0]):
    for j in range(patches_img.shape[1]):
        single_patch_img = patches_img[i, j, :, :, :]
        single_patch_img = scaler.fit_transform(single_patch_img.reshape(-1, single_patch_img.shape[-1])).reshape(single_patch_img.shape)
        pred = predict_patch(model, single_patch_img)
        patched_prediction.append(pred)

patched_prediction = np.array(patched_prediction)
patched_prediction = np.reshape(patched_prediction, [patches_img.shape[0], patches_img.shape[1], patch_size, patch_size])

unpatched_prediction = unpatchify(patched_prediction, (large_img.shape[0], large_img.shape[1]))


In [ ]:
# Convert predicted labels to RGB
prediction_rgb = label_to_rgb(unpatched_prediction)

In [ ]:
# save the predicted tif
output_path = "/content/drive/MyDrive/Semantic_segmentation_dataset2/output/predicted_mask_coordinates114.tif"


In [ ]:

with rasterio.open(
    output_path,
    'w',
    driver='GTiff',
    height=prediction_rgb.shape[0],
    width=prediction_rgb.shape[1],
    count=3,  # Number of channels
    dtype=prediction_rgb.dtype,
    crs=crs,  # Use the saved CRS information
    transform=transform  # Use the saved transform information
) as dst:
    dst.write(prediction_rgb[:, :, 0], 1)  # Red channel
    dst.write(prediction_rgb[:, :, 1], 2)  # Green channel
    dst.write(prediction_rgb[:, :, 2], 3)  # Blue channel

print(f"Predicted mask saved to {output_path}")


In [ ]:
def plot_segmented_image_with_legend(segmented_image):
    labels = ['Background', 'Building', 'Woodland', 'Water', 'Road']
    colors = [(0, 0, 0), (60, 16, 152), (132, 41, 246), (226, 169, 41), (128, 64, 128)]

In [ ]:
def label_to_rgb(predicted_image):
    Building = np.array([0, 0, 0])  # Background
    Building = np.array([60, 16, 152])  # Building
    Vegetation = np.array([132, 41, 246])  # Woodland
    Water = np.array([226, 169, 41])  # Water

    Unlabeled = np.array([155, 155, 155])

    segmented_img = np.zeros((predicted_image.shape[0], predicted_image.shape[1], 3), dtype=np.uint8)
    segmented_img[predicted_image == 0] = Building
    segmented_img[predicted_image == 1] = Vegetation
    segmented_img[predicted_image == 2] = Water
    segmented_img[predicted_image == 3] = Unlabeled
    return segmented_img

prediction_without_smooth_blending = label_to_rgb(unpatched_prediction)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
if img.dtype != np.uint8:
    img = img.astype(np.uint8)
if img.ndim == 2:  # Grayscale image
    img = np.stack((img,)*3, axis=-1)  # Convert to RGB by duplicating channels
elif img.ndim == 3 and img.shape[2] == 1:  # Single channel grayscale image
    img = np.concatenate([img]*3, axis=2)  # Convert to RGB
if img.shape[2] > 3:
    img = img[:, :, :3]
if prediction_rgb.ndim == 2:  # Grayscale prediction
    prediction_rgb = np.stack((prediction_rgb,)*3, axis=-1)  # Convert to RGB by duplicating channels
elif prediction_rgb.ndim == 3 and prediction_rgb.shape[2] == 1:  # Single channel prediction
    prediction_rgb = np.concatenate([prediction_rgb]*3, axis=2)  # Convert to RGB


if prediction_rgb.shape[2] > 3:
    prediction_rgb = prediction_rgb[:, :, :3]

# Display results
plt.figure(figsize=(12, 12))
plt.subplot(221)
plt.title('Testing Image')
plt.imshow(img)

plt.subplot(223)
plt.title('Prediction without smooth blending')
plt.imshow(prediction_rgb)

plt.show()

